# Preparação dos Dados UTA-RLDD

## Resumo
Este notebook processa o dataset UTA-RLDD para detecção de fadiga, extraindo características faciais para treinamento de modelos de deep learning.

### Características extraídas:
- **PERCLOS**: Percentual de fechamento das pálpebras
- **MAR**: Proporção de abertura da boca (detecta bocejos)
- **BLINK_RATE**: Taxa de piscadas por minuto
- **HEAD_STABILITY**: Estabilidade da postura da cabeça

### Resultados esperados:
- Sequências de 90 frames (3 segundos)
- ~32k sequências de 48 participantes
- Precisão esperada: 75-85%

In [ ]:
# Instalar dependências necessárias
!pip install --upgrade kagglehub
!pip install opencv-python mediapipe scikit-learn pandas numpy matplotlib seaborn scipy

In [ ]:
# Importar bibliotecas
import kagglehub
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import re
import json
import pickle
from pathlib import Path
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import euclidean
from scipy import signal
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Configurar MediaPipe
mp_face_mesh = mp.solutions.face_mesh

# Parâmetros do processamento
TARGET_FPS = 30
SEQUENCE_LENGTH = 90  # 3 segundos
OVERLAP_RATIO = 0.67  # 67% de sobreposição
MIN_FACE_CONFIDENCE = 0.7

print(f"🚀 Sistema de extração de características inicializado")
print(f"📊 Sequências: {SEQUENCE_LENGTH} frames ({SEQUENCE_LENGTH/TARGET_FPS:.1f}s)")
print(f"🔄 Sobreposição: {OVERLAP_RATIO*100:.0f}%")

## 1. Download do Dataset

In [ ]:
# Verificar se o dataset já existe
dataset_path = Path("/home/zhizhunu/.cache/kagglehub/datasets/rishab260/uta-reallife-drowsiness-dataset/versions/1")

if dataset_path.exists():
    print(f"✅ Dataset já disponível em: {dataset_path}")
else:
    print("📥 Fazendo download do dataset UTA-RLDD...")
    try:
        dataset_path = kagglehub.dataset_download("rishab260/uta-reallife-drowsiness-dataset")
        dataset_path = Path(dataset_path)
        print(f"✅ Dataset baixado para: {dataset_path}")
    except AttributeError:
        print("❌ Erro: Atualize o kagglehub com: pip install --upgrade kagglehub")
        raise

# Verificar estrutura do dataset
video_files = list(dataset_path.rglob("*.mp4")) + list(dataset_path.rglob("*.mov")) + list(dataset_path.rglob("*.m4v"))
print(f"📁 Encontrados {len(video_files)} arquivos de vídeo")

## 2. Extrator de Características de Fadiga

In [ ]:
class ExtratorCaracteristicasFadiga:
    """
    Extrai características de fadiga de vídeos usando MediaPipe
    """
    
    def __init__(self):
        self.face_mesh = mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=MIN_FACE_CONFIDENCE,
            min_tracking_confidence=0.5
        )
        
        # Thresholds calibrados para detecção
        self.EAR_FECHADO_THRESHOLD = 0.25
        self.EAR_PISCADA_THRESHOLD = 0.22
        self.MAR_BOCEJO_THRESHOLD = 0.6
        self.JANELA_PERCLOS_SEC = 60
        self.JANELA_PISCADA_SEC = 30
        
        # Índices dos landmarks do MediaPipe
        self.OLHO_ESQUERDO = [362, 385, 387, 263, 373, 380]
        self.OLHO_DIREITO = [33, 160, 158, 133, 153, 144]
        self.BOCA_EXTERNA = [61, 84, 17, 314, 405, 320, 375, 321]
        self.PONTOS_CABECA = [1, 2, 5, 4, 6, 168, 8, 9, 10, 151]
        
        self.resetar_buffers()
    
    def resetar_buffers(self):
        """Reseta buffers temporais para novo vídeo"""
        buffer_size = self.JANELA_PERCLOS_SEC * TARGET_FPS
        self.buffer_ear = deque(maxlen=buffer_size)
        self.buffer_mar = deque(maxlen=self.JANELA_PISCADA_SEC * TARGET_FPS)
        self.posicoes_cabeca = deque(maxlen=TARGET_FPS * 5)
        self.timestamps_piscadas = deque(maxlen=100)
        self.timestamps_frames = deque(maxlen=buffer_size)
        
        # Controle de estado
        self.ultimo_ear = None
        self.estado_piscada = False
        self.frames_fechados_consecutivos = 0
        self.contador_frames = 0
    
    def calcular_ear(self, landmarks_olho):
        """Calcula Eye Aspect Ratio"""
        # Distâncias verticais
        v1 = euclidean(landmarks_olho[1], landmarks_olho[5])
        v2 = euclidean(landmarks_olho[2], landmarks_olho[4])
        # Distância horizontal
        h = euclidean(landmarks_olho[0], landmarks_olho[3])
        
        if h < 1e-6:
            return 0.0
        
        ear = (v1 + v2) / (2.0 * h)
        return max(0.0, min(1.0, ear))
    
    def calcular_mar(self, landmarks_boca):
        """Calcula Mouth Aspect Ratio para detecção de bocejos"""
        # Distâncias verticais em diferentes posições da boca
        v1 = euclidean(landmarks_boca[2], landmarks_boca[6])
        v2 = euclidean(landmarks_boca[3], landmarks_boca[7])
        # Distância horizontal
        h = euclidean(landmarks_boca[0], landmarks_boca[1])
        
        if h < 1e-6:
            return 0.0
            
        mar = (v1 + v2) / (2.0 * h)
        return max(0.0, min(2.0, mar))
    
    def detectar_piscada(self, ear_atual, timestamp):
        """Detecta piscadas com validação temporal"""
        piscada_detectada = False
        
        if self.ultimo_ear is not None:
            # Transição de fechamento do olho
            if (self.ultimo_ear > self.EAR_PISCADA_THRESHOLD and 
                ear_atual <= self.EAR_PISCADA_THRESHOLD):
                self.frames_fechados_consecutivos = 1
                self.estado_piscada = True
            
            # Olho fechado (acumula frames fechados)
            elif ear_atual <= self.EAR_PISCADA_THRESHOLD and self.estado_piscada:
                self.frames_fechados_consecutivos += 1
            
            # Transição de abertura (piscada completa)
            elif (self.ultimo_ear <= self.EAR_PISCADA_THRESHOLD and 
                  ear_atual > self.EAR_PISCADA_THRESHOLD and 
                  self.estado_piscada and 
                  2 <= self.frames_fechados_consecutivos <= 15):
                piscada_detectada = True
                self.timestamps_piscadas.append(timestamp)
                self.estado_piscada = False
                self.frames_fechados_consecutivos = 0
        
        self.ultimo_ear = ear_atual
        return piscada_detectada
    
    def calcular_perclos(self, timestamp):
        """Calcula PERCLOS com janela temporal adequada"""
        if len(self.buffer_ear) < 10:
            return 0.0
        
        # Usar dados recentes para cálculo do PERCLOS
        ears_recentes = list(self.buffer_ear)[-min(len(self.buffer_ear), 
                                                    TARGET_FPS * 30):]
        
        frames_fechados = sum(1 for ear in ears_recentes if ear <= self.EAR_FECHADO_THRESHOLD)
        perclos = (frames_fechados / len(ears_recentes)) * 100
        
        return min(100.0, perclos)
    
    def calcular_taxa_piscadas(self, timestamp):
        """Calcula piscadas por minuto com validação temporal"""
        if len(self.timestamps_piscadas) < 1:
            return 0.0
        
        # Contar piscadas na janela recente
        inicio_janela = timestamp - self.JANELA_PISCADA_SEC
        piscadas_recentes = [t for t in self.timestamps_piscadas if t >= inicio_janela]
        
        if len(piscadas_recentes) == 0:
            return 0.0
        
        # Converter para piscadas por minuto
        tempo_decorrido = min(self.JANELA_PISCADA_SEC, 
                             max(timestamp - self.timestamps_piscadas[0], 1.0))
        if tempo_decorrido < 1.0:
            return 0.0
        
        piscadas_por_minuto = (len(piscadas_recentes) / tempo_decorrido) * 60
        return min(60.0, piscadas_por_minuto)
    
    def calcular_estabilidade_cabeca(self, landmarks, timestamp):
        """Calcula estabilidade do movimento da cabeça"""
        pontos_cabeca = [landmarks[i] for i in self.PONTOS_CABECA]
        
        # Calcular centro da cabeça
        centro_cabeca = np.mean(pontos_cabeca, axis=0)[:2]
        self.posicoes_cabeca.append((centro_cabeca, timestamp))
        
        if len(self.posicoes_cabeca) < TARGET_FPS:
            return 0.0
        
        # Calcular movimento na janela recente
        posicoes_recentes = [pos[0] for pos in self.posicoes_cabeca]
        array_posicoes = np.array(posicoes_recentes)
        
        # Calcular estabilidade como inverso da variância do movimento
        variancia_movimento = np.var(array_posicoes, axis=0).sum()
        estabilidade = 1.0 / (1.0 + variancia_movimento * 1000)
        
        return estabilidade
    
    def extrair_caracteristicas_frame(self, frame, timestamp):
        """Extrai todas as características de um único frame"""
        self.contador_frames += 1
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resultados = self.face_mesh.process(frame_rgb)
        
        if not resultados.multi_face_landmarks:
            return None
        
        landmarks = resultados.multi_face_landmarks[0]
        h, w = frame.shape[:2]
        
        # Converter para coordenadas em pixels
        landmarks_px = [(lm.x * w, lm.y * h) for lm in landmarks.landmark]
        
        # Calcular EAR
        olho_esquerdo = [landmarks_px[i] for i in self.OLHO_ESQUERDO]
        olho_direito = [landmarks_px[i] for i in self.OLHO_DIREITO]
        ear_esquerdo = self.calcular_ear(olho_esquerdo)
        ear_direito = self.calcular_ear(olho_direito)
        ear_medio = (ear_esquerdo + ear_direito) / 2.0
        
        # Armazenar EAR para cálculos temporais
        self.buffer_ear.append(ear_medio)
        self.timestamps_frames.append(timestamp)
        
        # Calcular características
        piscada_detectada = self.detectar_piscada(ear_medio, timestamp)
        perclos = self.calcular_perclos(timestamp)
        taxa_piscadas = self.calcular_taxa_piscadas(timestamp)
        
        # Cálculo do MAR
        landmarks_boca = [landmarks_px[i] for i in self.BOCA_EXTERNA]
        mar = self.calcular_mar(landmarks_boca)
        self.buffer_mar.append(mar)
        
        # Estabilidade da cabeça
        estabilidade_cabeca = self.calcular_estabilidade_cabeca(landmarks_px, timestamp)
        
        return {
            'PERCLOS': perclos,
            'MAR': mar,
            'BLINK_RATE': taxa_piscadas,
            'HEAD_STABILITY': estabilidade_cabeca,
            'timestamp': timestamp,
            'frame_count': self.contador_frames,
            'blink_detected': piscada_detectada,
            'EAR': ear_medio
        }

print("✅ Extrator de características criado")

## 3. Análise da Estrutura do Dataset

In [ ]:
def analisar_estrutura_dataset(dataset_path):
    """Analisa a estrutura do UTA-RLDD e cria mapeamento dos vídeos"""
    video_files = []
    extensoes = ['.mp4', '.mov', '.m4v', '.avi']
    
    for ext in extensoes:
        video_files.extend(list(dataset_path.rglob(f"*{ext}")))
        video_files.extend(list(dataset_path.rglob(f"*{ext.upper()}")))
    
    print(f"Encontrados {len(video_files)} arquivos de vídeo")
    
    # Criar mapeamento do dataset
    dados_videos = []
    
    for caminho_video in video_files:
        nome_arquivo = caminho_video.name.lower()
        
        # Extrair label do nome do arquivo
        if nome_arquivo.startswith('0.'):
            label, estado = 0, 'Alert'
        elif nome_arquivo.startswith('5.'):
            label, estado = 5, 'Low_Vigilant'
        elif nome_arquivo.startswith('10') or '10.' in nome_arquivo or '10_' in nome_arquivo:
            label, estado = 10, 'Drowsy'
        else:
            continue
        
        # Extrair ID do participante do caminho
        partes_caminho = str(caminho_video).split('/')
        id_participante = 'desconhecido'
        
        for parte in reversed(partes_caminho):
            if re.match(r'^\d{1,2}$', parte):
                id_participante = f"{int(parte):02d}"
                break
        
        if id_participante != 'desconhecido':
            dados_videos.append({
                'video_path': str(caminho_video),
                'filename': caminho_video.name,
                'subject_id': id_participante,
                'label': label,
                'state': estado
            })
    
    df = pd.DataFrame(dados_videos)
    
    print(f"\n📊 Análise do Dataset:")
    print(f"Vídeos válidos: {len(df)}")
    print(f"Participantes: {df['subject_id'].nunique()}")
    print(f"\nDistribuição de labels:")
    print(df['state'].value_counts())
    
    return df

# Analisar dataset
df_videos = analisar_estrutura_dataset(dataset_path)
print(f"\n✅ Encontrados {len(df_videos)} vídeos válidos de {df_videos['subject_id'].nunique()} participantes")

## 4. Processamento de Vídeos

In [ ]:
def processar_video(caminho_video, extrator, duracao_max_sec=300, 
                   threshold_qualidade=0.8, pular_frames=1):
    """Processa um vídeo extraindo características com controle de qualidade"""
    cap = cv2.VideoCapture(caminho_video)
    
    if not cap.isOpened():
        return [], f"Não foi possível abrir o vídeo: {caminho_video}"
    
    # Propriedades do vídeo
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duracao = total_frames / fps if fps > 0 else 0
    
    max_frames = min(total_frames, int(duracao_max_sec * fps)) if fps > 0 else total_frames
    fps_efetivo = fps / pular_frames if pular_frames > 1 else fps
    
    print(f"  📹 {Path(caminho_video).name}: {fps:.1f}fps→{fps_efetivo:.1f}fps, {duracao:.1f}s")
    
    # Resetar extrator
    extrator.resetar_buffers()
    
    lista_caracteristicas = []
    contador_frames = 0
    contador_processados = 0
    falhas_deteccao = 0
    
    while contador_frames < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Pular frames para acelerar processamento
        if contador_frames % pular_frames == 0:
            timestamp = contador_frames / fps if fps > 0 else contador_frames / TARGET_FPS
            
            # Redimensionar para processamento consistente
            height, width = frame.shape[:2]
            if width > 480:
                scale = 480 / width
                new_width, new_height = 480, int(height * scale)
                frame = cv2.resize(frame, (new_width, new_height))
            
            # Extrair características
            caracteristicas = extrator.extrair_caracteristicas_frame(frame, timestamp)
            
            if caracteristicas is not None:
                caracteristicas['frame_number'] = contador_frames
                lista_caracteristicas.append(caracteristicas)
            else:
                falhas_deteccao += 1
            
            contador_processados += 1
            
            # Atualização de progresso
            if contador_processados % 200 == 0:
                progresso = (contador_frames / max_frames) * 100
                taxa_sucesso = len(lista_caracteristicas) / contador_processados
                print(f"    Progresso: {progresso:.1f}% ({len(lista_caracteristicas)} características, {taxa_sucesso:.1%} sucesso)")
        
        contador_frames += 1
    
    cap.release()
    
    # Avaliação da qualidade
    taxa_sucesso = len(lista_caracteristicas) / contador_processados if contador_processados > 0 else 0
    
    if taxa_sucesso < threshold_qualidade:
        return [], f"Vídeo de baixa qualidade: {taxa_sucesso:.1%} taxa de detecção"
    
    print(f"  ✅ Processados {len(lista_caracteristicas)} frames ({taxa_sucesso:.1%} taxa de sucesso)")
    return lista_caracteristicas, None

print("✅ Funções de processamento de vídeo prontas")

## 5. Processamento de Todos os Vídeos

In [ ]:
import time

# Configurações de processamento
MODO_RAPIDO = False
MAX_VIDEOS = len(df_videos)
DURACAO_MAX = 240  # 4 minutos por vídeo
THRESHOLD_QUALIDADE = 0.7
PULAR_FRAMES = 1  # Processar todos os frames

def processar_video_seguro(info_video):
    """Processa um único vídeo com tratamento de erros"""
    idx, row = info_video
    
    # Criar extrator local
    extrator_local = ExtratorCaracteristicasFadiga()
    
    try:
        lista_caracteristicas, erro = processar_video(
            row['video_path'], 
            extrator_local,
            duracao_max_sec=DURACAO_MAX,
            threshold_qualidade=THRESHOLD_QUALIDADE,
            pular_frames=PULAR_FRAMES
        )
        
        if erro:
            return {'status': 'erro', 'idx': idx, 'error': erro, 'row': row}
        
        if len(lista_caracteristicas) >= SEQUENCE_LENGTH:
            return {
                'status': 'sucesso',
                'idx': idx,
                'video_path': row['video_path'],
                'filename': row['filename'],
                'subject_id': row['subject_id'],
                'label': row['label'],
                'state': row['state'],
                'features': lista_caracteristicas,
                'n_frames': len(lista_caracteristicas)
            }
        else:
            return {
                'status': 'insuficiente',
                'idx': idx,
                'error': f'Frames insuficientes: {len(lista_caracteristicas)}',
                'row': row
            }
    
    except Exception as e:
        return {'status': 'excecao', 'idx': idx, 'error': str(e), 'row': row}

# Processar vídeos
videos_processados = []
videos_falharam = []

print(f"🚀 PROCESSAMENTO COMPLETO DO DATASET")
print(f"📊 Processando: {MAX_VIDEOS}/{len(df_videos)} vídeos")
print(f"⚡ Configurações: {DURACAO_MAX}s max, qualidade {THRESHOLD_QUALIDADE:.1f}")

# Mostrar thresholds calibrados
extrator_teste = ExtratorCaracteristicasFadiga()
print(f"🔧 Thresholds calibrados:")
print(f"   EAR_FECHADO: {extrator_teste.EAR_FECHADO_THRESHOLD}")
print(f"   EAR_PISCADA: {extrator_teste.EAR_PISCADA_THRESHOLD}")

print(f"⏱️  Tempo estimado: {MAX_VIDEOS * 0.5 / 60:.0f}-{MAX_VIDEOS * 2 / 60:.0f} minutos")

tempo_inicio = time.time()

# Processar vídeos sequencialmente
for idx, (video_idx, row) in enumerate(df_videos.iterrows()):
    print(f"\n🎬 Processando vídeo {idx+1}/{MAX_VIDEOS}: {row['subject_id']} {row['state']} ({row['filename']})")
    
    resultado = processar_video_seguro((video_idx, row))
    
    if resultado['status'] == 'sucesso':
        videos_processados.append({
            'video_path': resultado['video_path'],
            'filename': resultado['filename'],
            'subject_id': resultado['subject_id'],
            'label': resultado['label'],
            'state': resultado['state'],
            'features': resultado['features'],
            'n_frames': resultado['n_frames']
        })
        
        # Verificação rápida das características nos primeiros vídeos
        caracteristicas = resultado['features']
        perclos_medio = np.mean([f['PERCLOS'] for f in caracteristicas])
        taxa_piscadas_medio = np.mean([f['BLINK_RATE'] for f in caracteristicas])
        
        print(f"  ✅ SUCESSO: {resultado['n_frames']} frames extraídos")
        if idx < 5:  # Mostrar estatísticas para os primeiros 5
            print(f"     PERCLOS: μ={perclos_medio:.1f}%")
            print(f"     BLINK_RATE: μ={taxa_piscadas_medio:.1f} bpm")
    else:
        videos_falharam.append({
            'video_path': resultado['row']['video_path'],
            'error': resultado['error']
        })
        print(f"  ❌ FALHOU: {resultado['error']}")
    
    # Atualização de progresso a cada 10 vídeos
    if (idx + 1) % 10 == 0:
        tempo_decorrido = time.time() - tempo_inicio
        tempo_restante = (MAX_VIDEOS - idx - 1) * (tempo_decorrido / (idx + 1))
        print(f"\n📊 Progresso: {idx+1}/{MAX_VIDEOS} ({(idx+1)/MAX_VIDEOS*100:.1f}%)")
        print(f"⏱️  Decorrido: {tempo_decorrido/60:.1f}min, Restante: ~{tempo_restante/60:.1f}min")
        print(f"✅ Sucesso: {len(videos_processados)}, ❌ Falhas: {len(videos_falharam)}")

tempo_total = time.time() - tempo_inicio
print(f"\n=== Processamento Completo ===")
print(f"⏱️  Tempo total: {tempo_total:.1f} segundos ({tempo_total/60:.1f} minutos)")
print(f"✅ Processados com sucesso: {len(videos_processados)} vídeos")
print(f"❌ Falharam: {len(videos_falharam)} vídeos")
if len(videos_processados) + len(videos_falharam) > 0:
    print(f"📈 Taxa de sucesso: {len(videos_processados)/(len(videos_processados)+len(videos_falharam))*100:.1f}%")

## 6. Análise da Qualidade das Características

In [ ]:
if len(videos_processados) > 0:
    print("🔍 Analisando qualidade das características...")
    
    # Coletar todas as características para análise
    todas_caracteristicas = []
    todos_labels = []
    todos_estados = []
    todos_participantes = []
    
    for dados_video in videos_processados:
        for caracteristicas in dados_video['features']:
            todas_caracteristicas.append([
                caracteristicas['PERCLOS'],
                caracteristicas['MAR'],
                caracteristicas['BLINK_RATE'],
                caracteristicas['HEAD_STABILITY']
            ])
            todos_labels.append(dados_video['label'])
            todos_estados.append(dados_video['state'])
            todos_participantes.append(dados_video['subject_id'])
    
    # Criar DataFrame para análise
    nomes_caracteristicas = ['PERCLOS', 'MAR', 'BLINK_RATE', 'HEAD_STABILITY']
    df_caracteristicas = pd.DataFrame(todas_caracteristicas, columns=nomes_caracteristicas)
    df_caracteristicas['Label'] = todos_labels
    df_caracteristicas['State'] = todos_estados
    df_caracteristicas['Subject'] = todos_participantes
    
    print(f"\n📊 Estatísticas das Características por Estado:")
    stats_por_estado = df_caracteristicas.groupby('State')[nomes_caracteristicas].agg(['mean', 'std'])
    print(stats_por_estado.round(3))
    
    # Verificar poder discriminativo
    print(f"\n🎯 Poder Discriminativo (Alert vs Drowsy):")
    dados_alert = df_caracteristicas[df_caracteristicas['State'] == 'Alert']
    dados_drowsy = df_caracteristicas[df_caracteristicas['State'] == 'Drowsy']
    
    for caracteristica in nomes_caracteristicas:
        media_alert = dados_alert[caracteristica].mean()
        media_drowsy = dados_drowsy[caracteristica].mean()
        std_combinado = np.sqrt((dados_alert[caracteristica].var() + dados_drowsy[caracteristica].var()) / 2)
        tamanho_efeito = abs(media_drowsy - media_alert) / std_combinado if std_combinado > 0 else 0
        
        print(f"  {caracteristica:15}: {tamanho_efeito:.3f} (Alert: {media_alert:.2f}, Drowsy: {media_drowsy:.2f})")
        
        if tamanho_efeito > 0.8:
            print(f"    ✅ Efeito grande - altamente discriminativo")
        elif tamanho_efeito > 0.5:
            print(f"    ✅ Efeito médio - boa discriminação")
        elif tamanho_efeito > 0.2:
            print(f"    ⚠️  Efeito pequeno - discriminação limitada")
        else:
            print(f"    ❌ Efeito muito pequeno - discriminação pobre")
    
    print(f"\n⏱️  Verificação de Variação Temporal:")
    print(f"Total de frames analisados: {len(df_caracteristicas):,}")
    print(f"Total de participantes: {df_caracteristicas['Subject'].nunique()}")
    
else:
    print("❌ Nenhum vídeo processado disponível para análise")

## 7. Criação de Sequências Temporais

In [ ]:
def criar_sequencias_temporais(videos_processados, tamanho_sequencia=SEQUENCE_LENGTH, 
                              razao_sobreposicao=OVERLAP_RATIO):
    """Cria sequências temporais com labeling realístico"""
    todas_sequencias = []
    todos_labels = []
    todos_participantes = []
    todas_infos_video = []
    
    nomes_caracteristicas = ['PERCLOS', 'MAR', 'BLINK_RATE', 'HEAD_STABILITY']
    passo = max(1, int(tamanho_sequencia * (1 - razao_sobreposicao)))
    
    print(f"Criando sequências: comprimento={tamanho_sequencia}, passo={passo} ({razao_sobreposicao*100:.0f}% sobreposição)")
    
    for dados_video in videos_processados:
        lista_caracteristicas = dados_video['features']
        
        if len(lista_caracteristicas) < tamanho_sequencia:
            continue
        
        # Converter para array
        array_caracteristicas = np.array([
            [f[nome] for nome in nomes_caracteristicas] 
            for f in lista_caracteristicas
        ])
        
        # Criar sequências com sobreposição
        sequencias_video = []
        for i in range(0, len(array_caracteristicas) - tamanho_sequencia + 1, passo):
            sequencia = array_caracteristicas[i:i + tamanho_sequencia]
            
            # Verificação de qualidade: garantir variação temporal
            stds_temporais = np.std(sequencia, axis=0)
            if np.any(stds_temporais > 0.001):
                sequencias_video.append(sequencia)
        
        if len(sequencias_video) > 0:
            todas_sequencias.extend(sequencias_video)
            todos_labels.extend([dados_video['label']] * len(sequencias_video))
            todos_participantes.extend([dados_video['subject_id']] * len(sequencias_video))
            todas_infos_video.extend([{
                'filename': dados_video['filename'],
                'state': dados_video['state']
            }] * len(sequencias_video))
        
        print(f"  {dados_video['subject_id']} {dados_video['state']}: {len(sequencias_video)} sequências")
    
    print(f"\n✅ Criadas {len(todas_sequencias)} sequências no total")
    
    return (
        np.array(todas_sequencias),
        np.array(todos_labels),
        np.array(todos_participantes),
        todas_infos_video
    )

if len(videos_processados) > 0:
    # Criar sequências
    X_raw, y, participantes, info_videos = criar_sequencias_temporais(videos_processados)
    
    print(f"\n📊 Resumo do Dataset de Sequências:")
    print(f"Formato: {X_raw.shape} (amostras, passos_tempo, características)")
    print(f"Labels: {len(np.unique(y))} classes")
    print(f"Participantes: {len(np.unique(participantes))}")
    
    # Distribuição de classes
    labels_unicos, contagens = np.unique(y, return_counts=True)
    print(f"\nDistribuição de classes:")
    for label, contagem in zip(labels_unicos, contagens):
        nome_classe = {0: 'Alert', 5: 'Low_Vigilant', 10: 'Drowsy'}.get(label, f'Classe_{label}')
        print(f"  {nome_classe} ({label}): {contagem:,} sequências ({contagem/len(y)*100:.1f}%)")
    
else:
    print("❌ Nenhum vídeo processado disponível para criação de sequências")

## 8. Normalização das Características

In [ ]:
if len(videos_processados) > 0:
    print("🔧 Aplicando normalização inteligente das características...")
    
    # Usar RobustScaler para lidar melhor com outliers
    scaler = RobustScaler()
    
    # Reorganizar para normalização
    n_amostras, tam_seq, n_caracteristicas = X_raw.shape
    X_reorganizado = X_raw.reshape(-1, n_caracteristicas)
    
    # Aplicar e transformar
    X_normalizado_flat = scaler.fit_transform(X_reorganizado)
    X_normalizado = X_normalizado_flat.reshape(n_amostras, tam_seq, n_caracteristicas)
    
    print(f"✅ Normalização das características concluída")
    
    # Analisar características normalizadas
    nomes_caracteristicas = ['PERCLOS', 'MAR', 'BLINK_RATE', 'HEAD_STABILITY']
    print(f"\n📊 Faixas das Características Normalizadas:")
    for i, nome_caracteristica in enumerate(nomes_caracteristicas):
        dados_caracteristica = X_normalizado[:, :, i].flatten()
        print(f"  {nome_caracteristica:15}: [{dados_caracteristica.min():6.2f}, {dados_caracteristica.max():6.2f}]")
        print(f"                     μ={dados_caracteristica.mean():6.3f}, σ={dados_caracteristica.std():6.3f}")
    
    print(f"\n✅ Análise da normalização das características completa")
else:
    print("❌ Nenhum dado disponível para normalização")

## 9. Configuração de Validação Cruzada por Participante

In [ ]:
if len(videos_processados) > 0:
    print("🔄 Criando divisões de validação cruzada por participante...")
    
    participantes_unicos = np.unique(participantes)
    n_participantes = len(participantes_unicos)
    
    print(f"Total de participantes: {n_participantes}")
    
    # Criar divisões de validação cruzada 5-fold
    n_folds = min(5, n_participantes)
    divisoes_cv = []
    
    if n_participantes >= 5:
        # Validação cruzada 5-fold padrão
        participantes_por_fold = n_participantes // n_folds
        
        for fold in range(n_folds):
            # Participantes de teste
            idx_inicio = fold * participantes_por_fold
            idx_fim = idx_inicio + participantes_por_fold
            if fold == n_folds - 1:  # Último fold pega os restantes
                idx_fim = n_participantes
            
            participantes_teste = participantes_unicos[idx_inicio:idx_fim]
            participantes_restantes = np.setdiff1d(participantes_unicos, participantes_teste)
            
            # Dividir restantes em treino/validação
            if len(participantes_restantes) >= 4:
                participantes_treino, participantes_val = train_test_split(
                    participantes_restantes, test_size=0.2, random_state=42
                )
            else:
                participantes_treino = participantes_restantes
                participantes_val = np.array([])
            
            # Criar máscaras
            mascara_treino = np.isin(participantes, participantes_treino)
            mascara_val = np.isin(participantes, participantes_val)
            mascara_teste = np.isin(participantes, participantes_teste)
            
            divisoes_cv.append({
                'fold': fold,
                'train_subjects': participantes_treino,
                'val_subjects': participantes_val,
                'test_subjects': participantes_teste,
                'train_mask': mascara_treino,
                'val_mask': mascara_val,
                'test_mask': mascara_teste
            })
            
            print(f"Fold {fold}: Treino({len(participantes_treino)}), Val({len(participantes_val)}), Teste({len(participantes_teste)})")
    
    else:
        # Divisão simples treino/teste para poucos participantes
        tamanho_teste = max(1, n_participantes // 4)
        participantes_treino, participantes_teste = train_test_split(
            participantes_unicos, test_size=tamanho_teste, random_state=42
        )
        
        mascara_treino = np.isin(participantes, participantes_treino)
        mascara_teste = np.isin(participantes, participantes_teste)
        
        divisoes_cv.append({
            'fold': 0,
            'train_subjects': participantes_treino,
            'val_subjects': np.array([]),
            'test_subjects': participantes_teste,
            'train_mask': mascara_treino,
            'val_mask': np.zeros(len(participantes), dtype=bool),
            'test_mask': mascara_teste
        })
        
        print(f"Divisão única: Treino({len(participantes_treino)}), Teste({len(participantes_teste)})")
    
    print(f"\n✅ Criadas {len(divisoes_cv)} divisões de validação cruzada")
else:
    divisoes_cv = []
    print("❌ Nenhum dado disponível para configuração CV")

## 10. Salvar Dataset Processado

In [ ]:
if len(videos_processados) > 0:
    # Criar diretório de saída
    diretorio_saida = Path('/home/zhizhunu/tcc/ia-fadiga-main2/processed_data_uta_rldd_CORRECTED')
    diretorio_saida.mkdir(exist_ok=True)

    print(f"💾 Salvando dataset processado em {diretorio_saida}...")

    # Salvar arrays principais
    np.save(diretorio_saida / 'X_sequences.npy', X_normalizado)
    np.save(diretorio_saida / 'y_labels.npy', y)
    np.save(diretorio_saida / 'subjects.npy', participantes)

    print(f"  ✅ Arrays principais salvos: X{X_normalizado.shape}, y{y.shape}, subjects{participantes.shape}")

    # Salvar scaler
    with open(diretorio_saida / 'feature_scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    # Salvar divisões de validação cruzada
    with open(diretorio_saida / 'cv_splits.pkl', 'wb') as f:
        pickle.dump(divisoes_cv, f)

    # Criar extrator temporário para valores de threshold
    extrator_temp = ExtratorCaracteristicasFadiga()

    # Salvar metadados
    metadados = {
        'dataset': 'UTA-RLDD-PROCESSADO',
        'versao': '1.0',
        'caracteristicas': nomes_caracteristicas,
        'tamanho_sequencia': SEQUENCE_LENGTH,
        'razao_sobreposicao': OVERLAP_RATIO,
        'fps_alvo': TARGET_FPS,
        'n_amostras': X_normalizado.shape[0],
        'n_caracteristicas': X_normalizado.shape[2],
        'n_participantes': len(participantes_unicos),
        'n_folds': len(divisoes_cv),
        'mapeamento_classes': {0: 'Alert', 5: 'Low_Vigilant', 10: 'Drowsy'},
        'videos_processados': len(videos_processados),
        'videos_falharam': len(videos_falharam),
        'configuracoes_processamento': {
            'modo_rapido': MODO_RAPIDO,
            'max_videos_processados': MAX_VIDEOS,
            'duracao_max_sec': DURACAO_MAX,
            'pular_frames': PULAR_FRAMES,
            'threshold_qualidade': THRESHOLD_QUALIDADE
        },
        'thresholds_caracteristicas': {
            'EAR_FECHADO': extrator_temp.EAR_FECHADO_THRESHOLD,
            'EAR_PISCADA': extrator_temp.EAR_PISCADA_THRESHOLD,
            'MAR_BOCEJO': extrator_temp.MAR_BOCEJO_THRESHOLD
        }
    }

    with open(diretorio_saida / 'metadata.json', 'w') as f:
        json.dump(metadados, f, indent=2)

    # Salvar resultados do processamento
    resultados = {
        'videos_processados': [{
            'filename': video['filename'],
            'subject_id': video['subject_id'],
            'state': video['state'],
            'label': video['label'],
            'n_frames': video['n_frames']
        } for video in videos_processados],
        'videos_falharam': videos_falharam,
        'metricas_qualidade': {
            'taxa_sucesso': len(videos_processados) / (len(videos_processados) + len(videos_falharam)),
            'media_frames_por_video': np.mean([v['n_frames'] for v in videos_processados]),
            'total_sequencias': len(X_normalizado)
        }
    }

    with open(diretorio_saida / 'processing_results.json', 'w') as f:
        json.dump(resultados, f, indent=2)

    # Salvar análise das características se disponível
    if 'df_caracteristicas' in locals():
        # Amostrar os dados se muito grandes
        if len(df_caracteristicas) > 100000:
            amostra_df = df_caracteristicas.sample(n=100000, random_state=42)
            amostra_df.to_csv(diretorio_saida / 'feature_analysis_sample.csv', index=False)
        else:
            df_caracteristicas.to_csv(diretorio_saida / 'feature_analysis.csv', index=False)

    # Resumo
    print(f"\n📁 Arquivos salvos:")
    for caminho_arquivo in sorted(diretorio_saida.glob('*')):
        if caminho_arquivo.is_file():
            tamanho_mb = caminho_arquivo.stat().st_size / (1024 * 1024)
            print(f"  - {caminho_arquivo.name}: {tamanho_mb:.2f} MB")

    print(f"\n✅ Dataset processado salvo com sucesso!")
else:
    print("❌ Nenhum dado para salvar")

## 11. Resumo Final

In [ ]:
print("=" * 80)
print("🎉 PREPARAÇÃO DOS DADOS UTA-RLDD - RESUMO FINAL")
print("=" * 80)

if len(videos_processados) > 0:
    print(f"\n✅ SUCESSO: Dataset de detecção de fadiga de alta qualidade criado!")
    print(f"\n📊 Estatísticas do Dataset:")
    print(f"  • Vídeos processados: {len(videos_processados)}/{len(df_videos)} ({len(videos_processados)/len(df_videos)*100:.1f}%)")
    print(f"  • Total de sequências: {X_normalizado.shape[0]:,}")
    print(f"  • Comprimento da sequência: {X_normalizado.shape[1]} frames ({X_normalizado.shape[1]/TARGET_FPS:.1f} segundos)")
    print(f"  • Características: {X_normalizado.shape[2]} ({', '.join(nomes_caracteristicas)})")
    print(f"  • Participantes: {len(participantes_unicos)}")
    print(f"  • Folds de validação cruzada: {len(divisoes_cv)}")
    
    # Distribuição de classes
    print(f"\n🎯 Distribuição de Classes:")
    labels_unicos, contagens = np.unique(y, return_counts=True)
    for label, contagem in zip(labels_unicos, contagens):
        nome_estado = {0: 'Alert', 5: 'Low_Vigilant', 10: 'Drowsy'}[label]
        percentual = contagem / len(y) * 100
        print(f"  • {nome_estado:12}: {contagem:6,} sequências ({percentual:5.1f}%)")
    
    # Performance esperada
    print(f"\n🚀 Performance Esperada do Modelo:")
    print(f"  • Precisão alvo: 75-85%")
    print(f"  • Características com dinâmica temporal adequada")
    print(f"  • Poder discriminativo aprimorado entre classes")
    print(f"  • Validação cruzada realística por participante")
    
    print(f"\n📂 Diretório de Saída:")
    print(f"  {diretorio_saida}")
    
    print(f"\n🔄 Próximos Passos:")
    print(f"  1. ✅ Dataset processado com características otimizadas")
    print(f"  2. 🚀 Treinar modelo TCN com dataset processado")
    print(f"  3. 📈 Comparar performance com resultados anteriores")
    print(f"  4. 🎯 Atingir meta de precisão de 75-85%")
    
else:
    print(f"\n❌ FALHOU: Nenhum vídeo foi processado com sucesso")
    print(f"\n🔧 Solução de problemas:")
    print(f"  • Verificar acessibilidade e formatos dos arquivos de vídeo")
    print(f"  • Verificar instalação do MediaPipe")
    print(f"  • Revisar mensagens de erro dos vídeos que falharam")
    
    if len(videos_falharam) > 0:
        print(f"\n❌ Vídeos que Falharam ({len(videos_falharam)}):")
        for i, falha in enumerate(videos_falharam[:5]):
            print(f"  {i+1}. {Path(falha['video_path']).name}: {falha['error']}")
        if len(videos_falharam) > 5:
            print(f"  ... e mais {len(videos_falharam)-5}")

print("\n" + "=" * 80)